In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob
import geopandas
from geodatasets import get_path

In [2]:
output_col_names = ['lon'
,'lat'
,'variable'
,'ipft'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [3]:
read_in_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/Aggregate_TEM/all_parallel_variable_info.csv')

In [4]:
read_in_data.input_group.unique()

array([    0,  1008, 10080, 10206, 10332, 10458, 10584, 10710, 10836,
       10962, 11088, 11214,  1134, 11340, 11466, 11592, 11718, 11844,
       11970, 12096, 12222, 12348, 12474,   126,  1260, 12600, 12726,
       12852, 12978, 13104, 13230, 13356, 13482, 13608, 13734,  1386,
       13860, 13986, 14112, 14238, 14364, 14490, 14616, 14742, 14868,
       14994,  1512, 15120, 15246, 15372, 15498, 15624, 15750, 15876,
       16002, 16128, 16254,  1638, 16380, 16506, 16632, 16758, 16884,
       17010, 17136, 17262, 17388, 17514,  1764, 17640, 17766, 17892,
       18018, 18144, 18270, 18396, 18522, 18648, 18774,  1890, 18900,
       19026, 19152, 19278, 19404, 19530, 19656, 19782, 19908, 20034,
        2016, 20160, 20286, 20412, 20538, 20664, 20790, 20916, 21042,
       21168, 21294,  2142, 21420, 21546, 21672, 21798, 21924, 22050,
       22176, 22302, 22428, 22554,  2268, 22680, 22806, 22932, 23058,
       23184, 23310, 23436, 23562, 23688, 23814,  2394, 23940, 24066,
       24192, 24318,

In [6]:
forest_vegs = [4, 5, 6, 8, 9, 10, 11, 16, 17, 18, 19, 20, 25, 33]
forest_types = ["Boreal Forest", "Forested Boreal Wetlands", "Boreal Woodlands","Mixed Temperate Forests", 
               "Temperate Coniferous Forests", "Temperate Deciduous Forests", "Temperate Forested Wetlands", 
               "Tropical Evergreen Forests", "Tropical Forested Wetlands", "Tropical Deciduous Forests", "Xeromorphic Forests and Woodlands"
               ,"Tropical Forested Floodplains", "Temperate Forested Floodplains", "Temperate Broadleaved Evergreen Forests"]

forest_pfts = pd.DataFrame({
'current_veg':forest_vegs, 
    'forest_type':forest_types
})


forest_subtypes = [4, 5, 6, 10, 12]
forest_subtypes_names = ["Boreal", "Temperate Coniferous", "Temperate Deciduous", "Tropical", "Temperate Broadleaved Evergreen Forests"]

forest_subtypes = pd.DataFrame({
'subtype':forest_subtypes, 
    'forest_type':forest_subtypes_names
})


In [13]:
forest_subtypes

,subtype,forest_type
0,4,Boreal
1,5,Temperate Coniferous
2,6,Temperate Deciduous
3,10,Tropical
4,12,Temperate Broadleaved Evergreen Forests


In [14]:
# all_output_vars = read_in_data['output_var'].unique()
# all_output_vars[4:len(all_output_vars)]
all_output_vars = ['NPP', 'SOILORGC', 'GPP', 'VEGINGPP',
        'VEGC', ]

In [16]:
####get soilorgc
for var in all_output_vars[1:3]:
    var_files = read_in_data.loc[read_in_data['output_var']==var].reset_index()
    print(var)
    # for i in range(2):
    for i in range(len(var_files)):
    ###read in cohort output
        cohort_output = pd.read_csv(var_files.loc[i, 'output_paths']
                    , names = output_col_names)

        # print('cohort rows read in : '+ str(len(cohort_output)))    
        cohort_output=cohort_output[['lon', 'lat','year','ipft', 'cohort_area','potential_veg' ,'subtype', 'annual_sum' ,'monthly_maximum' ,'monthly_mean' ,'monthly_minimum' ]].reset_index()

        #### match pfts to the current veg, bin by standage for forests
#         conditions = [
#          cohort_output['current_veg'].isin([50, 49, 52, 53, 54, 55, 56]) 
#             ,cohort_output['current_veg'].isin([51, 47])
#         , cohort_output['current_veg'].isin([48, 46]) 
#             ,cohort_output['current_veg'].isin([4, 5, 6, 8, 9, 
#                                                 10, 11, 16, 17, 18, 19, 20, 25, 33])
#             ,cohort_output['current_veg'].isin([7, 12, 13, 14, 22, 23, 24, 25, 26, 27, 28, 30, 31])
#             ,cohort_output['current_veg'].isin([15, 35, 29])
#             ,cohort_output['current_veg'].isin([2, 3])
#             ,cohort_output['current_veg'].isin([21])
#         ]

#         values = [
#                   'crop', 
#                   'pasture', 
#                   'urban or suburban', 
#                   'forest', 
#                   'grass', 
#                   'shrub', 
#                   'tundra',
#                     'desert']
#         cohort_output['pft'] = np.select(conditions, values, default = 'other')

        cohort_output['subtype'] = cohort_output['ipft'] + 1

        cohort_output=cohort_output.merge(forest_subtypes, on = 'subtype', how = 'left')

        # cohort_output.to_csv('/home/smmrrr/TEM_output_processed/Aggregate_TEM/'+var_files.loc[i, 'output_var']+str(var_files.loc[i, 'output_group'])+'.csv'
        #                     ,index = False)
        cohort_output = cohort_output.loc[cohort_output['subtype'].isin(forest_subtypes)]
        # print('forest only cohorts : '+ str(len(cohort_output)))    

        if (i == 0):
            all_cohort_output = cohort_output
        else:
            all_cohort_output = pd.concat([all_cohort_output, cohort_output], axis = 0)

        if i % 50 ==0:
            print(i)

        ####get weighted average
#         all_cohort_output['temp_weight'] = all_cohort_output['annual_sum'] * all_cohort_output['cohort_area'] 


#         all_cohort_output_sum = all_cohort_output.groupby(
#             ['lon','lat','year','pft','subtype', 'forest_type']
#         )[['temp_weight', 'cohort_area']].sum()

#         # all_grids =  all_grids.reset_index()

#         all_cohort_output_sum['value_weight'] = all_cohort_output_sum['temp_weight']/all_cohort_output_sum['cohort_area']
#         all_cohort_output_sum = all_cohort_output_sum.reset_index()
        # print('cohort rows post cohort collapse : '+ str(len(annual_cohort_output)))
        # all_cohort_output_sum
        

    all_cohort_output.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/Aggregate_TEM/condensed_variables_'+var+'.csv', index = False)
    # all_cohort_output_sum.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/Aggregate_TEM/condensed_variables_'+var+'.csv', index = False)
        # intervals_standage = [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 3000]
        # cohort_output['stand_age_bin'] = pd.cut(
        #     cohort_output['stand_age'], bins=intervals_standage)
        # # print(annual_current_veg[['pft', 'stand_age_bin']].value_counts())
        # cohort_output.loc[cohort_output['pft']!='forest', ['stand_age_bin']] =  cohort_output['stand_age_bin'].min()
        # cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.right).astype(int)

SOILORGC
0
50
100
150
200
250
300
350
400
450
GPP
0
50
100
150
200
250
300
350
400
450
